___

<a href='https://github.com/eliasmelul/'> <img src='https://s3.us-east-2.amazonaws.com/wordontheamazon.com/BlueLogoNoBackground.png' style='width: 12em;' align='right' /></a>
# Finding my Schitt's Creek
#### Data Collection: General Socioeconomic Data
___
<h3 align="right">by Elias Melul, Data Scientist </h3> 

___



## Collecting General Socioeconomic Data

---
**GOOD NEWS!** To use the recommender system, you don't need to rerun and scrap the web. I've made the data available for you **<a href='https://github.com/eliasmelul/finding_schitts/blob/master/Data/scraped_datausa.csv'>here</a>**.

Want direct access? **<a href='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/scraped_datausa.csv'>Click Here</a>**

---
This data includes the average monthly temperature of about 5800 cities and the monthy _swing_ - the difference between the average maximum temperature (F) and the average minimum temperature (F) every month.

----


<h2 id = "DataCollection">Data Collection</h2>
___

Since we've already collected the weather information, we are going to use said dataset to generate URLs that link us to the datausa.io paths for each city.


----
**What data will be collected and how?**

There is a CRAZY amount of data available on the web about all cities, but these can be very dispar and from numerous websites. Lucky me, I found [this](https://datausa.io/) website that contains a lot of the information I could use (thank you Deloitte and Datawheel!). I will then combine this information with weather information and FourSquare information about venues to complete the dataframe and begin modeling!

**Some of the variables scraped are:**

* Population and Population Change (Year to Year)
* Poverty Rate
* Median Age
* Median Household Income and Median Household Income Change (Year to Year)
* Number of Employees and Number of Employees Change (Year to Year)
* Median Property Value and Median Property Value Change (Year to Year)
* Average Male and Female Salary, and a ratio of Average Male to Female Salary
* Gini coefficient in 2017 and 2018, as well as it's change (Year to Year)
* Ratio of Patients to Clinicians (county-wise)
* Foreign-born population percentage
* Citizen population percentage
* Total degrees awarded in 2018 (higher education)
* Male to Female ratio of awarded degrees
* Number of degrees per capita
* Number of households in city
* Population per household (people per household)
* Homeownership Percentage (Rent vs Own)
* Average Commute Time (minutes)



#### Import Libraries

---
The libraries imported are not all used in this notebook. However, to be able to follow the whole project, please make sure you have these installed!

Otherwise use pip or conda to install them on your laptop or computer.

In [1]:
import pandas as pd # import pandas for dataframes
import numpy as np
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.display import HTML, display
from IPython.display import Image 
from IPython.core.display import HTML 

pd.options.display.max_columns = None
pd.options.display.max_rows=None


%matplotlib inline

**Import weather dataset**

---

Import weather data to generate URLs and scrap datausa.io

In [3]:
weatherCity_df = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/final_weather_data.csv', index_col=0)
weatherCity_df.head()

,State,City,State_Abb,CityState,Temp Jan,Temp Feb,Temp Mar,Temp Apr,Temp May,Temp Jun,Temp Jul,Temp Aug,Temp Sep,Temp Oct,Temp Nov,Temp Dec,Swing Jan,Swing Feb,Swing Mar,Swing Apr,Swing May,Swing Jun,Swing Jul,Swing Aug,Swing Sep,Swing Oct,Swing Nov,Swing Dec
0,Alabama,Addison,AL,"Addison, AL",39.5,43.5,51.5,59.5,67.5,74.5,78.5,78.0,71.5,60.5,51.5,42.0,21.0,21.0,23.0,23.0,23.0,21.0,21.0,22.0,23.0,25.0,23.0,20.0
1,Alabama,Alabaster,AL,"Alabaster, AL",44.0,48.5,55.5,62.5,70.5,77.5,81.0,80.0,74.5,64.0,55.0,46.5,20.0,21.0,23.0,23.0,21.0,19.0,20.0,20.0,21.0,22.0,22.0,19.0
2,Alabama,Alexander City,AL,"Alexander City, AL",43.5,47.0,54.5,61.5,69.5,77.0,80.0,79.0,73.5,63.0,54.0,45.5,23.0,24.0,27.0,27.0,25.0,22.0,22.0,22.0,23.0,26.0,26.0,23.0
3,Alabama,Aliceville,AL,"Aliceville, AL",43.5,47.0,55.5,63.0,70.5,78.0,81.0,80.5,74.5,64.0,54.5,45.5,25.0,26.0,27.0,28.0,27.0,24.0,22.0,23.0,25.0,28.0,27.0,25.0
4,Alabama,Andalusia,AL,"Andalusia, AL",46.5,50.0,56.5,62.5,70.5,77.5,79.5,79.5,75.0,65.0,56.0,49.0,29.0,30.0,31.0,31.0,29.0,25.0,25.0,23.0,26.0,30.0,32.0,28.0


**Generate a list of URLs that link us to datausa.io's path to each city**

---

For that, we will need a list of all the cities and their respective state abbreviation. Why? Because the path to our scrapping website is as follows:

    /city-name-state_abbretiation
    
    For example:
    /new-york-ny

---

However, not all URLs are that clear. Some contain paths that include 'metropolitan area' or some require a 'county'. In this case, we will generate URLs that also include the following form:

    /new-york-ny-metro-area

Once we generate these URLs, we will check if they are valid, extract another list of only valid URLs and use those to scrape data.

In [4]:
cities_list = weatherCity_df['City']+ "-"+weatherCity_df.State_Abb
cities_list = list(cities_list)
cities_list= [i.lower().replace(' ','-') for i in cities_list]

#Concatenate the domain to the generated path to recreate all the URLS
cities_metro = [i+'-metro-area' for i in cities_list]
cities_list = cities_list+cities_metro

urls_cities = ["https://datausa.io/profile/geo/"+i for i in cities_list]

**Checking the validity of each URL**

---

To prevent computational complications we decided to separate the URL validation and scraping tasks. However, combining these steps simply requires us to move the scraping functions prior to the next loop and include them in the calls where _exist = 1_.

In [8]:
# Now that we have all URLs, we should check whether these URLs are valid
#Check if url connections exist
checker = []
for url in urls_cities:
    c = requests.get(url)
    soup = BeautifulSoup(c.content, features='lxml')
    res = soup.find_all("h4",{"class":"pt-non-ideal-state-title"})
    if (len(res) > 0):
        exist = 0
    else:
        try:
            tes = soup.find("div",{"class":"content-container"})
            tes = tes.get_text()
            if (tes == 'N/A'):
                exist = 0
            else:
                exist =1
        except: 
            exist=1
            pass
           
    tempdic = {"URL":url,
              "Exist":exist}
    checker.append(tempdic)

checker = pd.DataFrame(checker)
checker.head()

,URL,Exist
0,https://datausa.io/profile/geo/addison-al,1
1,https://datausa.io/profile/geo/alabaster-al,1
2,https://datausa.io/profile/geo/alexander-city-al,1
3,https://datausa.io/profile/geo/aliceville-al,1
4,https://datausa.io/profile/geo/andalusia-al,1


In [9]:
existing = checker[checker.Exist==1]
print(f"The number of valid URLs is: {existing.shape[0]}")
nonexisting = checker[checker.Exist==0]
print(f"The number of invalid URLs is: {nonexisting.shape[0]}")

The number of valid URLs is: 10492
The number of invalid URLs is: 1440


While the number of valid URLs is much larger than the city count we have, some will be duplicates due to our URL generation patterns. We will deal with this issue later on.

In [10]:
#Create a list with the existing
urls_cities = existing['URL'].to_list()

In [11]:
# checkerer.to_csv('C:/Users/melul/Desktop/FMSC/Data/checker_df_urls.csv')
# existing.to_csv('C:/Users/melul/Desktop/FMSC/Data/existing.csv')
# nonexisting.to_csv('C:/Users/melul/Desktop/FMSC/Data/nonexisting.csv')

### Scraping dataUSA

For this section, we have defined everything as functions, so that we can easily iterate over all the URLs. 

---
The first function simply retrieves the text data from the HTML section specified by the class types and names.

---

The second function uses the first function to retrieve each metric wanted for each city as strings, and transforms such to the type desired. In this case, all the information is numeric except the name of the city. A dictionary will be returned.

In [12]:
def get_text_from_class(soup,classname,type1="div",type2="class"):
    city_content = soup.find(type1, {type2:classname})
    city_info = city_content.find_all('p')
    content_raw = []
    for i in city_info:
        i = str(i)
        removefirst = i[3:-4]
        content_raw.append(removefirst)
    return(content_raw)

In [13]:
def get_basic_city_info(url):
    html = requests.get(url=url)
    soup = BeautifulSoup(html.content, features='lxml')
    
    ###### Get content from dashboard
    #Name of the city
    city_name = soup.find("p").get_text()
    
    #Raw dashboard content
    content_raw = get_text_from_class(soup, "profile-stats")
    
    #Population of City
    Population = content_raw[1]
    
    if Population[-1:] == "M":
        Population = float(Population[0:-1])*1000000
    else:
        Population = float(Population.replace(',',''))
    
    #Population Change YTY
    Population_Change = content_raw[2]
    if Population_Change[-7:] == 'decline':
        Population_Change = float(Population_Change[0:-9])*-1
    else:
        Population_Change = float(Population_Change[0:-8])
        
    #Poverty Rate
    Poverty_Rate = float(content_raw[4][0:-1])
    
    #Median Age
    Median_Age = float(content_raw[7])
    
    #Median Household Income and Change
    Median_Household_Income = float(content_raw[10][1:].replace(',',''))
    Median_Household_Income_Change = content_raw[11]
    if Median_Household_Income_Change[-7:]=='decline':
        Median_Household_Income_Change = float(Median_Household_Income_Change[0:-9])*-1
    else:
        Median_Household_Income_Change = float(Median_Household_Income_Change[0:-8])
    
    #Number of Employees
    Number_Employees = content_raw[13]
    if Number_Employees[-1:] == "M":
        Number_Employees = float(Number_Employees[0:-1])*1000000
    else:
        Number_Employees = float(Number_Employees.replace(',',''))
    
    #Change Number of Employees
    Number_Employees_Change = content_raw[14]
    if Number_Employees_Change[-7:]=='decline':
        Number_Employees_Change = float(Number_Employees_Change[0:-9])*-1
    else:
        Number_Employees_Change = float(Number_Employees_Change[0:-8])
    
    #Median Property Value
    Median_Property_Value = content_raw[16][1:]
    if Median_Property_Value[-1:] == "M":
        Median_Property_Value = float(Median_Property_Value[0:-1])*1000000
    else:
        Median_Property_Value = float(Median_Property_Value.replace(',',''))
    
    #Median Property Value Change
    Median_Property_Value_Change = content_raw[17]
    if Median_Property_Value_Change[-7:]=='decline':
        Median_Property_Value_Change = float(Median_Property_Value_Change[0:-9])*-1
    else:
        Median_Property_Value_Change = float(Median_Property_Value_Change[0:-8])
    
    #Wage Distribution across genders
    wage_gender = get_text_from_class(soup, "topic income_gender TextViz")
    avg_male_salary = float(wage_gender[2][1:].replace(',',''))
    avg_female_salary = float(wage_gender[5][1:].replace(',',''))
    gender_salary_ratio = avg_male_salary/avg_female_salary
  
    #Gini coefficient distribution
    gini_coeff = get_text_from_class(soup, "topic income_distro TextViz")
    gini_2018 = float(gini_coeff[2])
    gini_2017 = float(gini_coeff[4])
    gini_change_percent = (gini_2018-gini_2017)*100/gini_2017
    
    #Health Ratio (Patients to Clinicians)
    health_ratio = get_text_from_class(soup, "topic clinician_patient_ratio TextViz")
    health_ratio = float(health_ratio[2].replace(' to 1','').replace(',',''))
    
    #Foreign Born Population
    foreign_ratio = get_text_from_class(soup, "topic foreign_born TextViz")
    foreign_ratio = float(foreign_ratio[1][:-1].replace(',',''))
    
    #Citizen Ratio
    citizen_ratio = get_text_from_class(soup, "topic citizenship TextViz")
    citizen_ratio = float(citizen_ratio[1][:-1].replace(',',''))
    
    #Degrees awareded
    # There seem to be some cities without education infomration... we need to handle this exception
    try:
        degrees_awarded = get_text_from_class(soup, "topic edu_gender TextViz")
        degrees_men = float(degrees_awarded[1].replace(',',''))
        degrees_women = float(degrees_awarded[3].replace(',',''))
        degrees_gender_ratio_M2F = degrees_men/degrees_women 
        total_degrees = degrees_men+degrees_women
        degrees_per_capita = total_degrees/Population
    except:
        total_degrees = 0
        degrees_gender_ratio_M2F = 1
        degrees_per_capita = None
    
    #Number of Households
    households = get_text_from_class(soup, "topic household_income TextViz")
    households = households[5]
    if households[-1:] == "M":
        households = float(households[0:-1])*1000000
    elif households[-1:]=="k":
        households = float(households[0:-1])*1000
    else:
        households = float(households.replace(',',''))
    people_per_house = Population/households
    
    #Rent vs Ownership of Homes
    rent_own = get_text_from_class(soup, "topic rent_own TextViz")
    rent_own = float(rent_own[1][:-1])
    
    #Communite time
    commute_time = get_text_from_class(soup, "topic commute_time TextViz")
    commute_time = float(commute_time[1][:-8])
    
    basic_information = {'City':city_name,
                        'Population':Population,
                        'Population Change':Population_Change,
                        'Poverty Rate':Poverty_Rate,
                        'Median Age':Median_Age,
                        'Median Household Income':Median_Household_Income,
                        'Median Household Income Change':Median_Household_Income_Change,
                        'Number Employees':Number_Employees,
                        'Number Employees Change':Number_Employees_Change,
                        'Median Property Value':Median_Property_Value,
                        'Median Property Value Change':Median_Property_Value_Change,
                        'Average Male Salary':avg_male_salary,
                        'Average Female Salary':avg_female_salary,
                        'Gender Salary Ratio M2F':gender_salary_ratio,
                        'Gini 2018':gini_2018,
                        'Gini Change':gini_change_percent,
                        'Patient to Clinician Ratio':health_ratio,
                        'Foreign Born Population Ratio':foreign_ratio,
                        'Citizens Percentage':citizen_ratio,
                        'Total Degrees':total_degrees,
                        'Degrees Ratio M2F':degrees_gender_ratio_M2F,
                        'Degrees per Capita':degrees_per_capita,
                        'Households':households,
                        'People Per House':people_per_house,
                        'Homeownership':rent_own,
                        'Commute Time':commute_time}
    
    
    return basic_information

##### Functions Use Example
---

We will use NYC to see how the functions work and how the output looks.

In [20]:
url = 'https://datausa.io/profile/geo/lancaster-pa-metro-area'
get_basic_city_info(url)

{'City': 'Lancaster, PA',
 'Population': 543557.0,
 'Population Change': 0.12,
 'Poverty Rate': 10.4,
 'Median Age': 38.7,
 'Median Household Income': 66277.0,
 'Median Household Income Change': 4.53,
 'Number Employees': 271712.0,
 'Number Employees Change': -1.2,
 'Median Property Value': 220500.0,
 'Median Property Value Change': 7.98,
 'Average Male Salary': 69779.0,
 'Average Female Salary': 52119.0,
 'Gender Salary Ratio M2F': 1.3388399623937528,
 'Gini 2018': 0.469,
 'Gini Change': -0.6355932203389837,
 'Patient to Clinician Ratio': 1232.0,
 'Foreign Born Population Ratio': 4.83,
 'Citizens Percentage': 97.4,
 'Total Degrees': 4990.0,
 'Degrees Ratio M2F': 0.6990125978890024,
 'Degrees per Capita': 0.009180269962487836,
 'Households': 202000.0,
 'People Per House': 2.6908762376237623,
 'Homeownership': 67.4,
 'Commute Time': 21.4}

**Now that we have defined these functions, lets iterate over all our valid URLs to extract the specified metrics**

---

In [21]:
cities_basic = []
for url in urls_cities:
    try:
        data = get_basic_city_info(url)
        cities_basic.append(data)
    except:
        try:
            data = get_basic_city_info(url)
            cities_basic.append(data)
        except:
            try:
                data = get_basic_city_info(url)
                cities_basic.append(data)
            except:
                pass

cities = pd.DataFrame(cities_basic)
print(cities.shape)
cities.head()

(4233, 26)


,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,Median Property Value Change,Average Male Salary,Average Female Salary,Gender Salary Ratio M2F,Gini 2018,Gini Change,Patient to Clinician Ratio,Foreign Born Population Ratio,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time
0,"Alabaster, AL",32567.0,0.923,11.1,37.0,74383.0,1.44,15674.0,-0.716,167800.0,0.902,58549.0,40615.0,1.441561,0.46,0.436681,1166.0,6.130,94.9,3.0,0.000000,0.000092,10800.0,3.015463,82.7,28.7
1,"Alexander City, AL",40756.0,-0.493,21.2,43.3,42181.0,7.07,15317.0,1.450,107000.0,1.710,58549.0,40615.0,1.441561,0.46,0.436681,1535.0,1.350,99.2,350.0,0.612903,0.008588,16400.0,2.485122,71.9,24.5
2,"Aliceville, AL",2466.0,-23.400,34.6,36.3,24097.0,20.90,810.0,-18.600,102800.0,5.440,58549.0,40615.0,1.441561,0.46,0.436681,2981.0,0.243,99.8,0.0,1.000000,NaN,928.0,2.657328,48.4,21.1
3,"Andalusia, AL",8918.0,-1.370,17.3,41.1,36101.0,6.14,3629.0,7.460,110400.0,12.800,58549.0,40615.0,1.441561,0.46,0.436681,1991.0,1.180,99.7,514.0,0.903704,0.057636,3610.0,2.470360,61.3,15.8
4,"Ashland, AL",2113.0,-5.290,24.6,45.7,27264.0,-12.20,868.0,1.640,133000.0,48.400,58549.0,40615.0,1.441561,0.46,0.436681,3389.0,2.980,97.0,0.0,1.000000,NaN,895.0,2.360894,50.8,24.1


When generating the URLs, we generalized to include paths like _/new-york-ny-metro-area_. This may cause an issue: duplicated cities in the dataset. However, the reason why we included this in the first place was to match the general city people know about rather than specific parts of it or boroughs. For example, in DATAUSA, _Lancaser, PA_ is not the metropolitan area that people typically think about (if you ever think about Lancaster), but rather the city/town of Lancaster. 

Therefore, we must delete all duplicated records and keep those that have the largest populations.

---
**Deleting Duplicates**

---

By sorting the data by population descending and reseting the index, we can now remove all duplicate rows based on their position. In other words, we can keep the first duplicate record - the metropolitan area one most likely.

In [28]:
dupli = cities[cities['City'].duplicated(keep=False)]
print(f"Shape of dataframe with duplicates: {dupli.shape}")
dupli = dupli.sort_values(by='Population', ascending=False)
dupli = dupli.reset_index().drop('index', axis=1)
dupli = dupli.drop_duplicates(subset='City',keep='first')
print(f"Shape after dropping duplicates: {dupli.shape}")

cities = cities.drop_duplicates(subset='City', keep=False)
print(f"Shape of complete dataset after dropping all duplicated rows, not just one: {cities.shape}")

cities = pd.concat([cities,dupli])
print(f"The final dataframe has shape: {cities.shape}")
dupli.head()

Shape of dataframe with duplicates: (0, 26)
Shape after dropping duplicates: (0, 26)
Shape of complete dataset after dropping all duplicated rows, not just one: (4050, 26)
The final dataframe has shape: (4050, 26)


,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,Median Property Value Change,Average Male Salary,Average Female Salary,Gender Salary Ratio M2F,Gini 2018,Gini Change,Patient to Clinician Ratio,Foreign Born Population Ratio,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time


In [30]:
cities.head()

,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,Median Property Value Change,Average Male Salary,Average Female Salary,Gender Salary Ratio M2F,Gini 2018,Gini Change,Patient to Clinician Ratio,Foreign Born Population Ratio,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time
0,"Alabaster, AL",32567.0,0.923,11.1,37.0,74383.0,1.44,15674.0,-0.716,167800.0,0.902,58549.0,40615.0,1.441561,0.46,0.436681,1166.0,6.130,94.9,3.0,0.000000,0.000092,10800.0,3.015463,82.7,28.7
1,"Alexander City, AL",40756.0,-0.493,21.2,43.3,42181.0,7.07,15317.0,1.450,107000.0,1.710,58549.0,40615.0,1.441561,0.46,0.436681,1535.0,1.350,99.2,350.0,0.612903,0.008588,16400.0,2.485122,71.9,24.5
2,"Aliceville, AL",2466.0,-23.400,34.6,36.3,24097.0,20.90,810.0,-18.600,102800.0,5.440,58549.0,40615.0,1.441561,0.46,0.436681,2981.0,0.243,99.8,0.0,1.000000,NaN,928.0,2.657328,48.4,21.1
3,"Andalusia, AL",8918.0,-1.370,17.3,41.1,36101.0,6.14,3629.0,7.460,110400.0,12.800,58549.0,40615.0,1.441561,0.46,0.436681,1991.0,1.180,99.7,514.0,0.903704,0.057636,3610.0,2.470360,61.3,15.8
4,"Ashland, AL",2113.0,-5.290,24.6,45.7,27264.0,-12.20,868.0,1.640,133000.0,48.400,58549.0,40615.0,1.441561,0.46,0.436681,3389.0,2.980,97.0,0.0,1.000000,NaN,895.0,2.360894,50.8,24.1


Awesome! We now have some general socioeconomic on 4050 cities.

While there were more cities in the weather dataset, due to scraping complications and error handling, some cities did not compute and are not included in the dataset. Nonetheless, these are smaller towns that should not affect us much in our project.